In [11]:
from tensorflow.keras.models import load_model,Model
from pydub import AudioSegment
import os
import random
import librosa
from spleeter.separator import Separator
import shutil
import gc
import tensorflow as tf
import numpy as np

In [12]:
def extract_instrumental(separator,audio,name):
    # Wait for implementation
    return ""

In [13]:
def extract_audio():
    # Wait for implementation
    return ""

In [14]:
def preprocess():
    # Wait for implementation
    return ""

In [15]:
def get_bpm(audio):
    return 20

In [16]:
def get_spectrogram(path):
    IMAGE_SIZE = 224
    img = tf.keras.preprocessing.image.load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255
    img_array = img_array.reshape(1,IMAGE_SIZE,IMAGE_SIZE,3)
    return img_array

In [17]:
def manipulate_audio(filepath):
    # Wait for implementation
    # extract_instrumental(audio)
    # extract_audio()
    
    return list(map(lambda x: "spectogram/Chill/" + x,os.listdir("spectogram/Chill/")))[:100]

In [18]:
def get_model():
    model_path = 'models/model_v2.h5'
    model = load_model(model_path)
    return model

In [19]:
def prediction(model,audio):
    class_labels = ["Anxious","Chill","Focus","Party","Romance","Sad"]
    predictions = model.predict(audio)
    mood = class_labels[np.argmax(predictions)]   
    feature_model = Model(model.input,model.layers[-6].output)
    features = feature_model.predict(audio)
    return mood,features

In [26]:
filepath = ""
model = get_model()
audios = manipulate_audio(filepath)
data = []
i = 0
for audio in audios:
    spectrogram = get_spectrogram(audio)
    bpm = get_bpm(audio)
    mood, features = prediction(model,spectrogram)
    data.append({
        "name":audio,
        "bpm":bpm,
        "mood":mood,
        "features":features.flatten().tolist()})
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [27]:
import json
json.dump(data,open("data.json","w"))